In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from diffusers import StableDiffusionImg2ImgPipeline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# create a custom dataset
class ProductDataset(Dataset):
    def __init__(self, image_dir, prompts, transform=None):
        self.image_dir = image_dir
        # ensuring the images are sorted by filename
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.prompts = prompts
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        prompt = self.prompts[idx]
        return {"image": image, "prompt": prompt}

In [6]:
image_directory = "/content/drive/My Drive/images_for_ai_image_gen"

prompts = [
    "3 ronnox socks grey green and pink colour options",      # 001.jpeg
    "pair of slippers with blue tropical print top",           # 002.jpeg
    "slipper with blue tropical print top",                    # 003.jpeg
    "female kid knee length blue dress",                       # 004.jpeg
    "female kid knee length blue dress",                       # 005.jpeg
    "blue mandela pattern baggy home pants",                   # 006.jpeg
    "blue mandela pattern baggy home pants",                   # 007.jpeg
    "dark blue compression socks"                              # 008.jpeg
]

In [7]:
dataset = ProductDataset(image_directory, prompts)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [9]:
# loading the img2img pipeline on GPU
model_id = "CompVis/stable-diffusion-v1-4"
img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    variant="fp16",
    torch_dtype=torch.float16
).to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
## putting a logo/design on top of product

init_image_path = "/content/drive/My Drive/images_for_ai_image_gen/008.jpeg"
init_image = Image.open(init_image_path).convert("RGB")

init_image = init_image.resize((512, 512))

prompt = "Put a pokemon on the middle of the sock"

with torch.no_grad():
    result = img2img_pipe(prompt=prompt, image=init_image, strength=0.75)
    modified_image = result.images[0]

# saving the modified image.
modified_image.save("/content/drive/My Drive/images_for_ai_image_gen/modified_008_pokemon.png")
print("Modified image saved as 'modified_008_pokemon.png'")

  0%|          | 0/37 [00:00<?, ?it/s]

Modified image saved as 'modified_008_pokemon.png'


In [12]:
## changing colour of product

init_image_path = "/content/drive/My Drive/images_for_ai_image_gen/008.jpeg"
init_image = Image.open(init_image_path).convert("RGB")

init_image = init_image.resize((512, 512))

prompt = "Change the colour of the socks to orange"

with torch.no_grad():
    result = img2img_pipe(prompt=prompt, image=init_image, strength=0.75)
    modified_image = result.images[0]

#saving the modified image
modified_image.save("/content/drive/My Drive/images_for_ai_image_gen/modified_008_colour.png")
print("Modified image saved as 'modified_008_colour.png'")

  0%|          | 0/37 [00:00<?, ?it/s]

Modified image saved as 'modified_008_colour.png'


In [13]:
## changing size of product

init_image_path = "/content/drive/My Drive/images_for_ai_image_gen/008.jpeg"
init_image = Image.open(init_image_path).convert("RGB")

init_image = init_image.resize((512, 512))

prompt = "Change the size of the socks to large"

with torch.no_grad():
    result = img2img_pipe(prompt=prompt, image=init_image, strength=0.75)
    modified_image = result.images[0]

# saving the modified image.
modified_image.save("/content/drive/My Drive/images_for_ai_image_gen/modified_008_size.png")
print("Modified image saved as 'modified_008_size.png'")

  0%|          | 0/37 [00:00<?, ?it/s]

Modified image saved as 'modified_008_size.png'


In [14]:
## adding text to the product

init_image_path = "/content/drive/My Drive/images_for_ai_image_gen/008.jpeg"
init_image = Image.open(init_image_path).convert("RGB")

init_image = init_image.resize((512, 512))

prompt = "Add the text ""Just do it"" to the socks"

with torch.no_grad():
    result = img2img_pipe(prompt=prompt, image=init_image, strength=0.75)
    modified_image = result.images[0]

# saving the modified image.
modified_image.save("/content/drive/My Drive/images_for_ai_image_gen/modified_008_text.png")
print("Modified image saved as 'modified_008_text.png'")

  0%|          | 0/37 [00:00<?, ?it/s]

Modified image saved as 'modified_008_text.png'


In [15]:
## changing material/texture of product

init_image_path = "/content/drive/My Drive/images_for_ai_image_gen/008.jpeg"
init_image = Image.open(init_image_path).convert("RGB")

init_image = init_image.resize((512, 512))

prompt = "change the material of the socks to cotton and add ruffled texture"

with torch.no_grad():
    result = img2img_pipe(prompt=prompt, image=init_image, strength=0.75)
    modified_image = result.images[0]

# saving the modified image.
modified_image.save("/content/drive/My Drive/images_for_ai_image_gen/modified_008_material.png")
print("Modified image saved as 'modified_008_material.png'")

  0%|          | 0/37 [00:00<?, ?it/s]

Modified image saved as 'modified_008_material.png'


In [16]:
### EVALUATING USING CLIP ###

In [17]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [18]:
# define the image and text prompt -- colour
image_path = "/content/drive/My Drive/images_for_ai_image_gen/modified_008_colour.png"
text_prompt = "Change the colour of the socks to orange"

image = Image.open(image_path)

inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)

logits_per_image = outputs.logits_per_image

clip_score = logits_per_image.item()
print("CLIP Score:", clip_score)

CLIP Score: 31.09707260131836


In [19]:
# define the image and text prompt -- size
image_path = "/content/drive/My Drive/images_for_ai_image_gen/modified_008_size.png"
text_prompt = "Change the size of the socks to large"

image = Image.open(image_path)

inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)

logits_per_image = outputs.logits_per_image

clip_score = logits_per_image.item()
print("CLIP Score:", clip_score)

CLIP Score: 26.21688461303711


In [20]:
# define the image and text prompt -- logo/design
image_path = "/content/drive/My Drive/images_for_ai_image_gen/modified_008_pokemon.png"
text_prompt = "Put a pokemon on the middle of the sock"

image = Image.open(image_path)

inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)

logits_per_image = outputs.logits_per_image

clip_score = logits_per_image.item()
print("CLIP Score:", clip_score)

CLIP Score: 31.78803062438965


In [21]:
# define the image and text prompt -- text customisation
image_path = "/content/drive/My Drive/images_for_ai_image_gen/modified_008_text.png"
text_prompt = "Add the text ""Just do it"" to the socks"
image = Image.open(image_path)

inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)

logits_per_image = outputs.logits_per_image

clip_score = logits_per_image.item()
print("CLIP Score:", clip_score)

CLIP Score: 26.055267333984375


In [22]:
# define the image and text prompt -- texture/material
image_path = "/content/drive/My Drive/images_for_ai_image_gen/modified_008_material.png"
text_prompt = "change the material of the socks to cotton and add ruffled texture"

image = Image.open(image_path)

inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)

logits_per_image = outputs.logits_per_image

clip_score = logits_per_image.item()
print("CLIP Score:", clip_score)

CLIP Score: 29.61479949951172


In [23]:
### EVALUATING USING LPIPS ###

In [24]:
import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [26]:
# loading the original and modified images - colour
original_image_path = '/content/drive/My Drive/images_for_ai_image_gen/008.jpeg'
modified_image_path = '/content/drive/My Drive/images_for_ai_image_gen/modified_008_colour.png'

original_image = Image.open(original_image_path).convert('RGB')
modified_image = Image.open(modified_image_path).convert('RGB')

img0 = transform(original_image).unsqueeze(0)
img1 = transform(modified_image).unsqueeze(0)

loss_fn = lpips.LPIPS(net='alex')

lpips_distance = loss_fn(img0, img1)

print("LPIPS Distance:", lpips_distance.item())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 65.7MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
LPIPS Distance: 0.16834697127342224


In [27]:
# loading the original and modified images - size
original_image_path = '/content/drive/My Drive/images_for_ai_image_gen/008.jpeg'
modified_image_path = '/content/drive/My Drive/images_for_ai_image_gen/modified_008_size.png'

original_image = Image.open(original_image_path).convert('RGB')
modified_image = Image.open(modified_image_path).convert('RGB')

img0 = transform(original_image).unsqueeze(0)
img1 = transform(modified_image).unsqueeze(0)

loss_fn = lpips.LPIPS(net='alex')

lpips_distance = loss_fn(img0, img1)

print("LPIPS Distance:", lpips_distance.item())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
LPIPS Distance: 0.16912418603897095


In [28]:
# loading the original and modified images - logo
original_image_path = '/content/drive/My Drive/images_for_ai_image_gen/008.jpeg'
modified_image_path = '/content/drive/My Drive/images_for_ai_image_gen/modified_008_pokemon.png'

original_image = Image.open(original_image_path).convert('RGB')
modified_image = Image.open(modified_image_path).convert('RGB')

img0 = transform(original_image).unsqueeze(0)
img1 = transform(modified_image).unsqueeze(0)

loss_fn = lpips.LPIPS(net='alex')

lpips_distance = loss_fn(img0, img1)

print("LPIPS Distance:", lpips_distance.item())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
LPIPS Distance: 0.31507667899131775


In [29]:
# loading the original and modified images - text based
original_image_path = '/content/drive/My Drive/images_for_ai_image_gen/008.jpeg'
modified_image_path = '/content/drive/My Drive/images_for_ai_image_gen/modified_008_text.png'

original_image = Image.open(original_image_path).convert('RGB')
modified_image = Image.open(modified_image_path).convert('RGB')

img0 = transform(original_image).unsqueeze(0)
img1 = transform(modified_image).unsqueeze(0)

loss_fn = lpips.LPIPS(net='alex')

lpips_distance = loss_fn(img0, img1)

print("LPIPS Distance:", lpips_distance.item())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
LPIPS Distance: 0.32690054178237915


In [30]:
# loading the original and modified images - material
original_image_path = '/content/drive/My Drive/images_for_ai_image_gen/008.jpeg'
modified_image_path = '/content/drive/My Drive/images_for_ai_image_gen/modified_008_material.png'

original_image = Image.open(original_image_path).convert('RGB')
modified_image = Image.open(modified_image_path).convert('RGB')

img0 = transform(original_image).unsqueeze(0)
img1 = transform(modified_image).unsqueeze(0)

loss_fn = lpips.LPIPS(net='alex')

lpips_distance = loss_fn(img0, img1)

print("LPIPS Distance:", lpips_distance.item())

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
LPIPS Distance: 0.20877929031848907
